torch.utils.data.DataLoader와 torch.utils.data.Dataset의 두 가지 데이터 기본 요소를 제공하여 미리 준비해둔 데이터셋 뿐만 아니라 가지고 있는 데이터를 사용할 수 있게 함 <br>
Dataset은 샘플과 정답을 저장 <br>
DataLoader는 Dataset을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체로 감쌈

데이터셋은 torch.utils.data.Dataset의 하위 클래스로 개별 데이터를 특정하는 함수가 구현

#  데이터셋 불러오기

* root 는 학습/테스트 데이터가 저장되는 경로

* train 은 학습용 또는 테스트용 데이터셋 여부를 지정

* download=True 는 root 에 데이터가 없는 경우 인터넷에서 다운로드

* transform 과 target_transform 은 특징(feature)과 정답(label) 변형(transform)을 지정

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)



데이터셋을 순회하고 시각화하기

# 파일에서 사용자 정의 데이터셋 만들기

사용자 정의 Dataset 클래스는 반드시 3개 함수를 구현해야 함
* '__init__'
* '__len__'
* '__getitem__'

FashionMNIST 이미지들은img_dir 디렉토리에 저장되고, 정답은 annotations_file csv 파일에 별도로 저장

In [3]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDatset(Dataset):
    # 학습을 할 때 데이터가 항상 알고리즘 학습에 필요한 최적의 처리가 된 형태로 제공되지 않기 때문에 적절한 Transform이 필요함
    # torchvision.datasets에서 매개변수 두 개를 받는데 하나는 feature을 변경하기 위한 transform이며 다른 하나는 label을 변경하기 위한 target_transform
    def __init__(self, annotations_file, img_dir, transform = None, target_transform = None):
        self.img_labels = pd.read_csv('annotations_file', names = ['file_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index):
        img_path = os.parh.join(self.img_dir, self.img_labels.iloc[index, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    

# '''___init___'''

__init__ 함수는 Dataset 객체가 생성될 때 한번만 실행. 여기서는 이미지와 주석 파일(annotation_file)이 포함된 디렉토리와 두 가지 변형을 초기화

# '''__len__'''

__len__ 함수는 데이터셋의 샘플 개수를 반환

# '''__getitem__'''

__getitem__ 함수는 주어진 인덱스에 해당하는 샘플을 데이터셋에서 불러오고 반환. <br>인덱스를 기반으로, 디스크에서 이미지의 위치를 식별하고, read_image 를 사용하여 이미지를 텐서로 변환하고, self.img_labels 의 csv 데이터로부터 해당하는 정답(label)을 가져오고, (해당하는 경우) 변형(transform) 함수들을 호출한 뒤, 텐서 이미지와 라벨을 Python 사전(dict)형으로 반환

# DataLoader로 학습용 데이터 준비하기

Dataset은 데이터셋의 특징을 가져오고, 하나의 샘플에 정답(label)을 지정하는 일을 한 번에 함. 모델을 학습할 때, 일반적으로 샘플들을 미니배치로 전달하고, 매 에폭마다 데이터를 다시 섞어서 과적합을 막고, 파이썬의 muliporcessing을 사용하여 데이터 검색 속도를 높이려고 함

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 64, shuffle = True)

# DataLoader를 통해 순회(iterate)

DataLoader에 데이터셋을 불러온 뒤에는 필요에 따라 데이터셋을 순회할 수 있음. 아래의 각 순회는 각 미니배치에 포함하는 train_features와 train_labels의 묶음을 반환 <br>
shuffle = True로 지정했으므로, 모든 배치를 순회한 뒤 데이터가 섞임

In [ ]:
# 이미지와 정답(label)을 표시합니다.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")


Feature batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])


: 